In [1]:
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

In [2]:
import os
load_dotenv() 
gemini_api_key = os.getenv("GEMINI_API_KEY")

if not gemini_api_key:
	raise ValueError("GEMINI_API_KEY not found in environment variables. Please set it in your .env file.")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.2, google_api_key=gemini_api_key)

In [3]:
persist_directory = "./chroma_db"

# Check if the directory exists
if not os.path.exists(persist_directory):
    # If it doesn't exist, create it
    print(f"Creating directory: {persist_directory}")
    os.makedirs(persist_directory)

    pdf_reader = PyPDFLoader("Attention_is_all_you_need.pdf")
    documents = pdf_reader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,)
    chunks = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    db = Chroma.from_documents(documents=chunks, embedding=embeddings,persist_directory=persist_directory)

else:
    print(f"Directory already exists: {persist_directory}")

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

Directory already exists: ./chroma_db


/var/folders/fg/8pnzznxs5n3_dv51ftv04hs00000gn/T/ipykernel_77412/3432414550.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/praveennair/Learning/Gen AI/RAG - Research Paperr Chatbot/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/fg/8pnzznxs5n3_dv51ftv04hs00000gn/T/ipykernel_77412/3432414550.py:22: LangChainDeprecationWarning: The class `Chroma` was depr

In [4]:

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Behave as a AI tutor, and respond the queries from the document trained, Given the following conversation and a follow question, rephrase the follow up question to be a standalone question.
                                                        Chat History:{chat_history}
                                                        Follow up Input: {question}
                                                        Standalone question:""")
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(), condense_question_prompt=CONDENSE_QUESTION_PROMPT, return_source_documents=True,
                                           verbose=False)

In [6]:
chat_history=[]
query="""What is an encoder?"""
result = qa.invoke({"question":query, "chat_history":chat_history})

print(result['answer'])

/Users/praveennair/Learning/Gen AI/RAG - Research Paperr Chatbot/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


The encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence of continuous representations z = (z1, ..., zn). The encoder is composed of a stack of N = 6 identical layers. Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-wise fully connected feed-forward network.
